# Bio2Py: Implementation Example 2
### Evaluation of Chemical Consumption for Physicochemical Phosphorus Removal

The following simple example illustrates how the user can combine Bio2Py auxiliary functions to automate diffrent actions in BioWin that are not defined on first hand as main functions in Bio2Py. It also demonstrates how Bio2Py can enhance data acquisition processes. 

In this example, we explore a physicochemical phosphorus removal system using FeCl3, within a specified influent. NaOH is used for adjusting pH.
The goal was to generate response surfaces for effluent TP and pH in relation to FeCl3 and NaOH consumption. This analysis could assist in determining the optimal combination of chemicals for influents with particular characteristics. 

***Steps***
1. Define a range of FeCl3 and NaOH flow rates to test.
2. Randomly select N (FeCl3, NaOH) pairs within these ranges.
3. Run N simulations in BioWin for the correspondig (FeCl3, NaOH) pais. --> This can be automated using Bio2Py auxiliary functions!
4. Save simulation results for each (FeCl3, NaOH) pair. --> This can be automated using Bio2Py auxiliary functions!
5. Visualize the results to understand how P and pH vary with FeCl3 and NaOH.

***Benefits of Bio2Py***

Bio2Py simplifies the execution of numerous simulations. In this example, 300 simulations were conducted by the API, enabling the generation of response surfaces for effluent TP and pH regarding FeCl3 and NaOH consumption. Analysis of these surfaces shows that various combinations of FeCl3 and NaOH achieve comparable levels of P removal.  other criteria such as minimizing costs, could be considered to determine the most suitable chemical consumption for the effluent plant.
Manually performing these simulations would be laborious and prone to data transfer errors.

This example utilizes matplotlib and plotly to visualize BioWin simulation results. Users can install these libraries using pip:
    
    pip install set_matplotlib
    pip install plotly
    pip install nbformat

**Remember**

Bio2Py automatices the process of loading influent data, changing parameters and running simulations in BioWin. It is necessary to set the system configuration layout, project options, unit system, report options, etc before using Bio2Py.  

In order to use Bio2Py, **BioWin simulation window must be fully visible**, as Bio2Py relies on image recognition to perform its tasks. 

Placing BioWin window on the left side of the screen is recomended. 

**Before using Bio2Py**:
- Open BioWin simulation file --> Implementation Example 2 (in this case)
- Set the zoom to 100% on BioWin window.
- Manually run a single flow balance and steady state/dynamic simulation. Check for any warning messages that could interrupt Bio2Py process. If necessary, the user may consider disabling BioWin alarms to prevent Bio2Py interruption.  
- Choose locations (if applicable): 
    - For saving BioWin generated reports with simulation results (File -> Report to Excel (TM) -> Choose directory).
    - From where variable influent data will be loaded (Influent icon -> Open file -> Choose directory)(***). Only necessary if running simulations with variable influent. Choose the same filepath arg .
- Place BioWin simulation window on the left side of the screen. 
- Make sure influent icon is visible. 

## Load libraries and set the environment

In [1]:
import Bio2Py
import pandas as pd
import pyautogui
import time
import pyperclip
import numpy as np
import matplotlib.pyplot as plt

In [2]:
filepath=r"C:\Users\Usuario\OneDrive - Facultad de Ingeniería\Maestría\Biowin - python\Case study\Implementation example 2" #change

In [3]:
simulation_window,influen_window=Bio2Py.setting_the_environment()

## Define functions for changing FeCl3 flow and NaOH flow 

Use Bio2Py auxiliary functions

**Important:**
Make sure you have dowloaded all the images contained in the 'Implementation Example 2' and save them in the same folder as this notebook.

In [3]:
def load_FeCl3 (flow):
    
    decimal_separator=Bio2Py.get_decimal_separator()

    pyautogui.doubleClick(pyautogui.locateCenterOnScreen('FeCl3.PNG',confidence=0.7, grayscale=True))
    time.sleep(2)
    pyautogui.click(pyautogui.locateCenterOnScreen('edit.PNG',confidence=0.7, grayscale=True))
    time.sleep(2)
    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('enter')
    
    if decimal_separator == ',':
        formatted_value = str(flow).replace('.', ',') 
        pyautogui.write(str(formatted_value), interval=0.05)
    elif decimal_separator == '.':
        pyautogui.write(str(flow), interval=0.05)
    else: 
        print('Error while getting decimal separator')    
    
    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('enter')
    time.sleep(1)
    pyautogui.press('\t')
    pyautogui.press('enter')

def load_influent_state_variable(flow):
    
    decimal_separator=Bio2Py.get_decimal_separator()

    pyautogui.doubleClick(pyautogui.locateCenterOnScreen('NaOH_5N.PNG',confidence=0.7, grayscale=True))
    time.sleep(2.5)
    pyautogui.click(pyautogui.locateCenterOnScreen('edit.PNG',confidence=0.65, grayscale=True))
    time.sleep(2)
    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('enter')
    
    if decimal_separator == ',':
        formatted_value = str(flow).replace('.', ',') 
        pyautogui.write(str(formatted_value), interval=0.05)
    elif decimal_separator == '.':
        pyautogui.write(str(flow), interval=0.05)
    else: 
        print('Error while getting decimal separator')    

    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('\t')
    pyautogui.press('enter')
    time.sleep(1)
    pyautogui.press('\t')
    pyautogui.press('enter')

## Generating Random FeCl3 and NaOH Pairs

1. Define a range of FeCl3 and NaOH flow rates to test 
2. Randomly select N (FeCl3, NaOH) pairs within these ranges.

In [4]:
random_state=55
n_points= 370
x=np.random.randint(400, 1200, n_points)
y=np.random.randint(0, 1750, n_points)

## Run simulations in BioWin and save results using Bio2Py

In [8]:
results=pd.DataFrame([])

for n in range(n_points):
    #Load chemical flows 
    load_FeCl3(x[n])
    time.sleep(0.5)
    load_influent_state_variable(y[n])
    time.sleep(1)
    
    #Run steady state simulation
    Bio2Py.steady_state('None', 'current',60) 
    
    #Save results
    time.sleep(1)
    Bio2Py.save_results('table','P',0,filepath)
    effluent=pd.read_csv('P_0.csv')
    TP_effluent=effluent.iloc[0,1]
    pH_effluent=effluent.iloc[0,3]
    results_iteration=effluent
    results_iteration['FeCl3(L/d)']=x[n]
    results_iteration['NaOH(L/d)']=y[n]
    results = pd.concat([results, results_iteration], axis=0)
    
    print(f'FeCl3 (L/d): {x[n]}:')
    print(f'NaOH (L/d): {y[n]}')
    print(f'TP (mg-P/L): {TP_effluent}')
    print(f'pH: {pH_effluent}')
    
    time.sleep(1)

00:02:13.062: Steady State


FeCl3 (L/d): 799:
NaOH (L/d): 1200
TP (mg-P/L): 4.03
pH: 6.73


00:02:44.415: Steady State


FeCl3 (L/d): 919:
NaOH (L/d): 1042
TP (mg-P/L): 3.57
pH: 5.46


00:03:15.578: Steady State


...

03:48:20.948: Steady State


FeCl3 (L/d): 557:
NaOH (L/d): 1003
TP (mg-P/L): 8.29
pH: 7.78


03:49:13.056: Steady State


FeCl3 (L/d): 1034:
NaOH (L/d): 606
TP (mg-P/L): 23.5
pH: 2.5
FeCl3 (L/d): 1141:
NaOH (L/d): 906
TP (mg-P/L): 23.5
pH: 2.5


ImageNotFoundException: 

In [ ]:
#results.to_csv('results_1.csv', index=False)

## If the user prefers not to execute all iterations, they have the option to load our results instead:

In [11]:
results=pd.read_csv('results_1.csv')
results

,Elements,P - Total P [mgP/L],P - Soluble PO4-P [mgP/L],pH [],COD - Total [mg/L],N - Total Kjeldahl Nitrogen [mgN/L],Total suspended solids [mg/L],FeCl3(L/d),NaOH(L/d)
0,Effluent,4.96,2.14,6.96,56.33,3.07,54.82,944,1600
1,Effluent,4.58,1.72,5.72,56.33,3.07,44.98,680,589
2,Effluent,6.23,3.56,3.30,56.33,3.07,55.79,979,1006
3,Effluent,14.96,13.28,2.64,56.33,3.07,47.62,841,357
4,Effluent,24.56,23.96,9.62,56.33,3.07,35.91,626,1616
...,...,...,...,...,...,...,...,...,...
365,Effluent,5.84,3.12,7.01,56.33,3.07,44.54,680,1072
366,Effluent,11.55,9.48,2.78,56.33,3.07,60.03,1137,1101
367,Effluent,4.09,1.18,5.98,56.33,3.07,55.31,949,1218
368,Effluent,19.61,18.45,2.41,56.33,3.07,46.18,845,32


## Plotting the results

In [14]:
import plotly.graph_objs as go
import numpy as np

x = results['FeCl3(L/d)']
y = results['NaOH(L/d)']
z = results['P - Total P [mgP/L]']

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                   marker=dict(size=5))])

fig.update_layout(title='Effluent TP as a function of FeCl3 and NaOH consumption',
                  scene=dict(
                      xaxis_title='FeCl3 (L/d)',
                      yaxis_title='NaOH 5N (L/d)',
                      zaxis_title='TP (mg-P/L)'
                  ))

fig.update_layout(width=800, height=600)

fig.show()


In [13]:
x = results['FeCl3(L/d)']
y = results['NaOH(L/d)']
z = results['pH []']

# Create the interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                   marker=dict(size=5))])

# Update the layout of the plot with axis titles
fig.update_layout(title='Effluent pH as a function of FeCl3 and NaOH consumption',
                  scene=dict(
                      xaxis_title='FeCl3 (L/d)',
                      yaxis_title='NaOH 5N (L/d)',
                      zaxis_title='pH'
                  ))

# Adjust the size of the plot
fig.update_layout(width=800, height=600)

# Show the plot
fig.show()


## Identify the FeCl3 and NaOH combinations that ensure compliance with local regulatory standards for effluent quality: 

- Filter rows where the 'TP' (Total Phosphorus) is less than 5 and the 'pH' is greater than or equal to 6. 
- Note that other parameters such as NTK and DQO already meet the local regulatory constraints (*).

(*) Decreto 253/979. Uruguay, 1979.

In [16]:
compliant_effluent_df =results[(results['P - Total P [mgP/L]'] < 5) & (results['pH []'] >= 6)]

In [18]:
x = results['FeCl3(L/d)']
y = results['NaOH(L/d)']
z = results['P - Total P [mgP/L]']

a = compliant_effluent_df['FeCl3(L/d)']
b = compliant_effluent_df['NaOH(L/d)']
c = compliant_effluent_df['P - Total P [mgP/L]']


fig=go.Figure()
fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                   marker=dict(size=5),name='Effluent'))

fig.add_trace(go.Scatter3d(x=a, y=b, z=c, mode='markers',
                                   marker=dict(size=5),name='Compliant effluent'))

fig.update_layout(title='Effluent TP as a function of FeCl3 and NaOH consumption',
                  scene=dict(
                      xaxis_title='FeCl3 (L/d)',
                      yaxis_title='NaOH (L/d)',
                      zaxis_title='TP (mg-P/L)'
                  ))
fig.update_layout(width=800, height=600)

fig.show()




With the presented plot we can observe that different combinations of FeCl3 and NaOH can render the effluent compliant with local regulations. Additionally, other criteria, such as minimizing costs, could be considered to determine the most suitable chemical consumption for the effluent plant.

Performing 370 simulations manually may seem daunting, but it can be easily achieved with Bio2Py.